In [ ]:
!wget https://www.techgig.com/files/DataScienceData/344575/Phising_Training_Dataset.csv
!wget https://www.techgig.com/files/DataScienceData/344575/Phising_Testing_Dataset.csv
!wget https://www.techgig.com/files/DataScienceData/344575/sample_submission.csv

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# !pip install sweetviz
import sweetviz as eda
import seaborn as sns

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# set pandas options
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

# /content/drive/MyDrive/TechGig_HPE_ML_2022
root = "/content/drive/MyDrive/TechGig_HPE_ML_2022/%s"

from joblib import dump, load

In [76]:
# read the data
train = pd.read_csv("./Phising_Training_Dataset.csv")
testdf = pd.read_csv("./Phising_Testing_Dataset.csv")
bestdf = pd.read_csv("./submission_07-best.csv")

In [55]:
train_unik = train.drop_duplicates()

In [56]:
train_unik.drop(columns=["key", "Result"], inplace=True)

In [57]:
test = testdf.drop(columns=["key"])

In [58]:
train_unik.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8955 entries, 0 to 8954
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   having_IP                    8955 non-null   int64
 1   URL_Length                   8955 non-null   int64
 2   Shortining_Service           8955 non-null   int64
 3   having_At_Symbol             8955 non-null   int64
 4   double_slash_redirecting     8955 non-null   int64
 5   Prefix_Suffix                8955 non-null   int64
 6   having_Sub_Domain            8955 non-null   int64
 7   SSLfinal_State               8955 non-null   int64
 8   Domain_registeration_length  8955 non-null   int64
 9   Favicon                      8955 non-null   int64
 10  port                         8955 non-null   int64
 11  HTTPS_token                  8955 non-null   int64
 12  Request_URL                  8955 non-null   int64
 13  URL_of_Anchor                8955 non-null   int

In [59]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   having_IP                    2100 non-null   int64
 1   URL_Length                   2100 non-null   int64
 2   Shortining_Service           2100 non-null   int64
 3   having_At_Symbol             2100 non-null   int64
 4   double_slash_redirecting     2100 non-null   int64
 5   Prefix_Suffix                2100 non-null   int64
 6   having_Sub_Domain            2100 non-null   int64
 7   SSLfinal_State               2100 non-null   int64
 8   Domain_registeration_length  2100 non-null   int64
 9   Favicon                      2100 non-null   int64
 10  port                         2100 non-null   int64
 11  HTTPS_token                  2100 non-null   int64
 12  Request_URL                  2100 non-null   int64
 13  URL_of_Anchor                2100 non-null   int

In [60]:
test.columns.values, train_unik.columns.values

(array(['having_IP', 'URL_Length', 'Shortining_Service',
        'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
        'having_Sub_Domain', 'SSLfinal_State',
        'Domain_registeration_length', 'Favicon', 'port', 'HTTPS_token',
        'Request_URL', 'URL_of_Anchor', 'Links_in_tags', 'SFH',
        'Submitting_to_email', 'Abnormal_URL', 'Redirect', 'on_mouseover',
        'RightClick', 'popUpWidnow', 'Iframe', 'age_of_domain',
        'DNSRecord', 'web_traffic', 'Page_Rank', 'Google_Index',
        'Links_pointing_to_page', 'Statistical_report'], dtype=object),
 array(['having_IP', 'URL_Length', 'Shortining_Service',
        'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
        'having_Sub_Domain', 'SSLfinal_State',
        'Domain_registeration_length', 'Favicon', 'port', 'HTTPS_token',
        'Request_URL', 'URL_of_Anchor', 'Links_in_tags', 'SFH',
        'Submitting_to_email', 'Abnormal_URL', 'Redirect', 'on_mouseover',
        'RightClick',

In [61]:
not_found = []
def check_record(record):
    c = train_unik['having_IP'] == record['having_IP']
    for col in train_unik.columns:
        c = c & (train_unik[col] == record[col])
    l = train_unik[c]
    
    if len(l) > 0:
        # display(l)
        r = train.loc[l.index, "Result"].values[0]
        # display(r)
        return r
    else:
        not_found.append(record)
        # assign -99 to undefined records
        return -99

trace_df = test.apply(check_record, axis=1)
display(trace_df)
"Not found", len(not_found)

0        1
1       -1
2       -1
3       -1
4        1
        ..
2095     1
2096   -99
2097    -1
2098   -99
2099   -99
Length: 2100, dtype: int64

('Not found', 653)

In [87]:
# prepare submit df
submitdf_raw = pd.DataFrame(dict(key=testdf.key, Result=trace_df))
submitdf = submitdf_raw.copy()
# assign undefinable records from best df
submitdf.loc[submitdf["Result"] == -99, "Result"] = bestdf[submitdf.Result == -99]["Result"]

submitdf.to_csv("compare_mat.csv", index=None)

In [65]:
submitdf[submitdf.Result == -99].shape

(653, 2)

In [98]:
submitdf.shape[0]*.05

105.0

In [91]:
testdf["Result"] = trace_df
testdf["Result2"] = trace_df
testdf.sample(5, random_state=42)

,key,having_IP,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result,Result2
1034,22372,1,-1,1,1,1,1,1,1,1,1,1,1,-1,1,-1,-1,1,1,0,1,1,-1,1,1,1,1,-1,1,0,1,1,1
1176,22514,-1,-1,1,1,1,-1,0,-1,-1,1,1,1,-1,-1,0,-1,1,1,1,1,1,1,1,-1,1,-1,1,1,1,1,-99,-99
67,21405,1,-1,1,1,1,-1,-1,-1,1,1,1,1,-1,-1,-1,-1,1,1,0,1,1,1,1,-1,-1,-1,-1,-1,1,1,-1,-1
1330,22668,-1,-1,1,1,1,-1,1,0,1,1,1,1,-1,0,-1,1,1,1,1,1,1,1,-1,-1,1,-1,-1,1,1,1,-99,-99
650,21988,-1,-1,-1,1,-1,1,0,1,-1,1,1,-1,1,0,0,-1,1,-1,1,1,1,1,1,1,-1,1,-1,1,1,1,1,1


From **Pearson-R** correlation we have seen below variables are highly correlated to the target variable: 
1. `SSLfinal_State`
1. `URL_of_Anchor`
1. `Prefix_Suffix`

In [93]:
# Find the no. of phising records in Train data
train[(train["Result"] == -1) & ((train["SSLfinal_State"] == -1) | (train["URL_of_Anchor"] == -1) | (train["Prefix_Suffix"] == -1)) ].shape

(3921, 32)

# Compare and count no. of records in Train dataset, where any of the attributes are -1

In [83]:
# Compare and count no. of records in Train data where
sum(train[(train["Result"] == -1) & ((train["SSLfinal_State"] == -1) | (train["URL_of_Anchor"] == -1) | (train["Prefix_Suffix"] == -1)) ].index == train[train["Result"] == -1].index)

3921

- Looks like the indices are good
- After performing simple data analysis, we can clearly see the key indicator of phishing is:
1. `SSLfinal_State`
1. `URL_of_Anchor`
1. `Prefix_Suffix`

- Mark `Result` as `Phishing` for certain record where any of the flags are `Phising=-1`

In [84]:
# undefined records
testdf.loc[ (testdf["Result2"] == -99), "Result2"].shape

(653,)

In [85]:
# undefined records where any of the flags is -1
testdf.loc[((testdf["SSLfinal_State"] == -1) | (testdf["URL_of_Anchor"] == -1) | (testdf["Prefix_Suffix"] == -1)) & (testdf["Result2"] == -99), "Result2"].shape

(584,)

In [86]:
# assign -1 to undefined records where any of the flags is -1
testdf.loc[((testdf["SSLfinal_State"] == -1) | (testdf["URL_of_Anchor"] == -1) | (testdf["Prefix_Suffix"] == -1)) & (testdf["Result2"] == -99), "Result2"] = -1 

In [74]:
# undefined records
testdf.loc[ (testdf["Result2"] == -99), "Result2"].shape

(69,)

In [88]:
# assign -1 to undefined records from bestdf
testdf.loc[testdf["Result2"] == -99, "Result2"] = bestdf[testdf["Result"] == -99]["Result"]


In [89]:
# undefined records
testdf.loc[ (testdf["Result2"] == -99), "Result2"].shape

(0,)

In [90]:
submit = testdf[["key", "Result"]]
submit.to_csv("compare_mat2.csv", index=None)

In [53]:
# submitdf[submitdf["Result"] != -99] == testdf[submitdf["Result"] != -99]

ValueError: Can only compare identically-labeled DataFrame objects

# Manually compare 653

In [108]:
u99 = submitdf_raw[submitdf_raw['Result'] == -99]
u99.shape

(653, 2)

In [104]:
# filter out the records
u99t = testdf[submitdf_raw['Result'] == -99]

In [109]:
# apply best scores for better comparison
u99t["Result"] = submitdf["Result"]

C:\Users\DNDLSS~1\AppData\Local\Temp/ipykernel_4456/1676988293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  u99t["Result"] = submitdf["Result"]


In [110]:
u99t.head()

,key,having_IP,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result,Result2
19,21357,1,-1,1,1,1,-1,-1,-1,-1,1,1,1,-1,-1,0,-1,1,1,0,1,1,1,1,-1,-1,-1,-1,1,0,1,-1,-99
54,21392,1,-1,1,1,1,-1,1,-1,1,-1,-1,1,-1,0,-1,-1,-1,1,0,-1,1,-1,-1,1,-1,1,-1,1,1,-1,-1,-99
79,21417,1,-1,1,1,1,-1,1,-1,1,1,1,1,-1,-1,-1,-1,1,1,0,1,1,1,1,1,1,-1,-1,-1,0,1,-1,-99
97,21435,1,-1,1,1,1,-1,-1,1,-1,-1,-1,1,-1,0,1,-1,-1,1,0,-1,1,-1,-1,-1,-1,0,-1,1,0,-1,-1,-99
106,21444,1,-1,1,1,1,-1,0,0,-1,1,1,1,-1,-1,0,-1,1,1,0,1,1,1,1,1,-1,-1,-1,-1,0,1,-1,-99


In [120]:
u99t.loc[((testdf["SSLfinal_State"] == -1) 
          | (u99t["URL_of_Anchor"] == -1)
          | (u99t["web_traffic"] == -1)
          | (u99t["Links_in_tags"] == -1)
          | (u99t["Prefix_Suffix"] == -1)), "Result"].shape

(613,)

In [119]:
u99t.loc[((testdf["SSLfinal_State"] != -1) 
          & (u99t["URL_of_Anchor"] != -1)
          & (u99t["web_traffic"] != -1)
          & (u99t["Links_in_tags"] != -1)
          & (u99t["Prefix_Suffix"] != -1)), "Result"].shape

(40,)

In [126]:
u99t[u99t.Result == 1].shape

(294, 33)